In [ ]:
!pip install google-generativeai gradio pandas openpyxl -q

print("✅ Installation complete!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.

# Data Loading & EDA

In [ ]:
import pandas as pd

df = pd.read_excel('/kaggle/input/thedata-set/input.xlsx')
df_labeled = df[df['الأهمية'].notna()]

print("="*70)
print("📊 DATA OVERVIEW")
print("="*70)
print(f"\n✓ Total records: {len(df):,}")
print(f"✓ Labeled records: {len(df_labeled):,}")
print(f"✓ Unlabeled records: {len(df) - len(df_labeled):,}")

print(f"\n{'='*70}")
print("📈 SEVERITY DISTRIBUTION")
print("="*70)
severity_counts = df_labeled['الأهمية'].value_counts()
for severity, count in severity_counts.items():
    percentage = (count / len(df_labeled)) * 100
    bar = "█" * int(percentage / 2)
    print(f"{severity:8s}: {count:3d} ({percentage:5.1f}%) {bar}")

print(f"\n{'='*70}")
print("📝 SAMPLE DATA")
print("="*70)
for idx, row in df_labeled.head(5).iterrows():
    text = row['النص'][:60] + "..." if len(row['النص']) > 60 else row['النص']
    severity = row['الأهمية']
    emoji = {'عادي': '🟢', 'متوسط': '🟡', 'هام': '🔴', 'عاي': '❓'}
    print(f"\n{emoji.get(severity, '❓')} {severity:8s} | {text}")

print(f"\n{'='*70}")
print("✅ Data loaded successfully!")
print("="*70)

📊 DATA OVERVIEW

✓ Total records: 1,000
✓ Labeled records: 1,000
✓ Unlabeled records: 0

📈 SEVERITY DISTRIBUTION
متوسط   : 825 ( 82.5%) █████████████████████████████████████████
عالي    : 130 ( 13.0%) ██████
عادي    :  45 (  4.5%) ██

📝 SAMPLE DATA

🟡 متوسط    | في شخص بلغ عن واحد يحاول يفتح سيارات الناس في بريدة .

❓ عالي     | بلاغ اتجار بالبشر

🟡 متوسط    | تم تسجيل مركبة تمشي عكس السير في الخرج .

🟡 متوسط    | تم تسجيل مشاجرة عند البقالة في الخرج .

🟢 عادي     | فيه سيارة موقفة بنص الشارع في مكة .

✅ Data loaded successfully!


# Classification System

In [3]:
def classify_simple(text):
    text_lower = text.lower()
    
    high_keywords = [
        'اختطاف', 'خطف', 'قتل', 'مقتل', 'وفيات', 'وفاة', 'اتجار', 
        'حريق', 'إطلاق نار', 'اطلاق نار', 'مسلح', 'جريمة', 'طلق نار',
        'انفجار', 'تفجير', 'إرهاب', 'ارهاب', 'طعن', 'ذبح'
    ]
    
    medium_keywords = [
        'مشاجرة', 'مشاجره', 'مضاربة', 'عراك', 'سرقة', 'سرقه', 
        'سرعة عالية', 'عكس السير', 'تخريب', 'اعتداء', 'تفحيط',
        'مشبوه', 'يصرخ', 'صراخ', 'تجمهر', 'حادث', 'انقلاب',
        'يحاول يفتح', 'يفتح سيارات', 'مخالفة'
    ]
    
    normal_keywords = [
        'موقفة', 'متوقفة', 'عطل', 'صوت عالي', 'ضوضاء'
    ]
    
    for keyword in high_keywords:
        if keyword in text_lower:
            return {'severity': 'High', 'confidence': 0.85, 'reasoning': f'Critical: {keyword}'}
    
    for keyword in medium_keywords:
        if keyword in text_lower:
            return {'severity': 'Medium', 'confidence': 0.75, 'reasoning': f'Moderate: {keyword}'}
    
    for keyword in normal_keywords:
        if keyword in text_lower:
            return {'severity': 'Normal', 'confidence': 0.70, 'reasoning': f'Minor: {keyword}'}
    
    return {'severity': 'Medium', 'confidence': 0.60, 'reasoning': 'Default classification'}

print("="*70)
print("✅ ENHANCED RULE-BASED CLASSIFIER")
print("="*70)
print(f"\n✓ Type: Keyword-based")
print(f"✓ High keywords: 18")
print(f"✓ Medium keywords: 18")
print(f"✓ Normal keywords:5")
print(f"✓ Fast & Simple")

print(f"\n{'='*70}")
print("🧪 TESTING")
print("="*70)

tests = ["بلاغ اختطاف طفل", "سيارة متوقفة", "مشاجرة في مطعم", "في شخص يصرخ بالشارع"]
for text in tests:
    result = classify_simple(text)
    emoji = {'Normal': '🟢', 'Medium': '🟡', 'High': '🔴'}
    print(f"\n{emoji[result['severity']]} {text[:40]}...")
    print(f"   → {result['severity']} ({result['confidence']:.0%})")

print(f"\n{'='*70}")
print("✅ Classifier ready!")
print("="*70)

✅ ENHANCED RULE-BASED CLASSIFIER

✓ Type: Keyword-based
✓ High keywords: 18
✓ Medium keywords: 18
✓ Normal keywords:5
✓ Fast & Simple

🧪 TESTING

🔴 بلاغ اختطاف طفل...
   → High (85%)

🟢 سيارة متوقفة...
   → Normal (70%)

🟡 مشاجرة في مطعم...
   → Medium (75%)

🟡 في شخص يصرخ بالشارع...
   → Medium (75%)

✅ Classifier ready!


# Testing

In [4]:
print("="*70)
print("📊 TESTING ON REAL DATA")
print("="*70)

samples = df_labeled.sample(10, random_state=42)
results = []

for idx, row in samples.iterrows():
    text = row['النص']
    actual = row['الأهمية']
    
    result = classify_simple(text)
    predicted = result['severity']
    
    emoji_actual = {'عادي': '🟢', 'متوسط': '🟡', 'هام': '🔴', 'عاي': '❓'}
    emoji_pred = {'Normal': '🟢', 'Medium': '🟡', 'High': '🔴'}
    
    match = "✅" if (
        (actual == 'عادي' and predicted == 'Normal') or
        (actual == 'متوسط' and predicted == 'Medium') or
        (actual == 'هام' and predicted == 'High')
    ) else "❌"
    
    print(f"\n{match} Text: {text[:50]}...")
    print(f"   Actual: {emoji_actual.get(actual, '❓')} {actual:8s} | Predicted: {emoji_pred[predicted]} {predicted:8s}")
    print(f"   Confidence: {result['confidence']:.0%}")
    
    results.append({
        'actual': actual,
        'predicted': predicted,
        'match': match == "✅"
    })

accuracy = sum(1 for r in results if r['match']) / len(results) * 100

print(f"\n{'='*70}")
print("📈 RESULTS SUMMARY")
print("="*70)
print(f"\nTotal tested: {len(results)}")
print(f"Correct: {sum(1 for r in results if r['match'])}")
print(f"Accuracy: {accuracy:.1f}%")
print(f"\n{'='*70}")

📊 TESTING ON REAL DATA

❌ Text: في شخص بلغ عن صوت طلق نار سمعته بالحي في الدمام ....
   Actual: 🟡 متوسط    | Predicted: 🔴 High    
   Confidence: 85%

✅ Text: جاني بلاغ عن مضاربة بين شباب عند الحديقة في المدين...
   Actual: 🟡 متوسط    | Predicted: 🟡 Medium  
   Confidence: 75%

✅ Text: لاحظت انقلاب سيارة عند الدائري في الرياض ....
   Actual: 🟡 متوسط    | Predicted: 🟡 Medium  
   Confidence: 75%

❌ Text: بلاغ قضية قتل...
   Actual: ❓ عالي     | Predicted: 🔴 High    
   Confidence: 85%

✅ Text: بلغوني عن مخالفة قطع إشارة بشارع الجامعة في المدين...
   Actual: 🟡 متوسط    | Predicted: 🟡 Medium  
   Confidence: 75%

✅ Text: صار تجمهر شباب عند المدرسة في أبها ....
   Actual: 🟡 متوسط    | Predicted: 🟡 Medium  
   Confidence: 75%

✅ Text: في شخص بلغ عن واحد يصرخ بالشارع بشكل مريب في مكة ....
   Actual: 🟡 متوسط    | Predicted: 🟡 Medium  
   Confidence: 75%

✅ Text: وصلنا بلاغ عن واحد يصرخ بالشارع بشكل مريب في جدة ....
   Actual: 🟡 متوسط    | Predicted: 🟡 Medium  
   Confidence: 75%

✅ Text: جاني

#  User Interface

In [5]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from datetime import datetime
import json

try:
    from openai import OpenAI
    openai_available = True
except:
    openai_available = False

history = []
unclear_reports = []
stats = {
    'emergency_field': 0,
    'patrol_required': 0,
    'contact_first': 0,
    'electronic_report': 0,
    'complaint': 0,
    'unclear': 0
}

severity_stats = {
    'High': 0,
    'Medium': 0,
    'Normal': 0,
    'Unclear': 0
}

api_key_input = widgets.Password(
    value='',
    placeholder='أدخل OpenAI API Key هنا (اختياري)',
    description='',
    layout=widgets.Layout(width='100%')
)

def classify_with_llm(text, api_key):
    try:
        client = OpenAI(api_key=api_key)
        
        prompt = f"""أنت نظام ذكي لتصنيف البلاغات الأمنية في السعودية. قم بتحليل البلاغ التالي وتصنيفه بدقة عالية.

البلاغ: "{text}"

قم بتصنيف البلاغ واعطني النتيجة بصيغة JSON فقط بهذا الشكل:
{{
    "severity": "High" أو "Medium" أو "Normal" أو "Unclear",
    "confidence": رقم من 0.0 إلى 1.0,
    "type": "وصف نوع البلاغ",
    "category": "التصنيف",
    "action": "emergency_field" أو "patrol_required" أو "contact_first" أو "electronic_report" أو "complaint" أو "unclear"
}}

⚠️ معايير التصنيف - انتبه جيداً:

**High (عالي الخطورة) - تحتاج مباشرة فورية:**
- أي وفاة أو قتيل أو متوفى
- اختطاف، قتل، طعن، ضرب بسلاح
- إطلاق نار، مسلح، انفجار
- حريق (أي حريق يعتبر خطير)
- اتجار بالبشر
- ابتزاز خطير، تحرش منتهي

**Medium (متوسط) - تحتاج دورية:**
- مشاجرة، مضاربة، ضرب عادي
- سرقة، سطو
- حوادث مرورية بدون وفيات
- تفحيط، سرعة عالية، دهس خفيف
- سائق متهور، عكس السير

**Normal (عادي) - تواصل أولاً:**
- سيارة متوقفة، موقفة، متعطلة
- شكوى عامة
- ضوضاء، إزعاج

**Unclear:**
- نص قصير جداً (أقل من 5 كلمات)
- غير مفهوم

الإجراءات:
- emergency_field: High severity - مباشرة ميدانية فورية
- electronic_report: ابتزاز، اتجار بالبشر، جرائم إلكترونية
- patrol_required: Medium severity - إرسال دورية
- contact_first: Normal severity - تواصل أولاً
- complaint: شكاوى عامة
- unclear: غير واضح

⚠️ مهم جداً: أي ذكر لـ "وفاة" أو "متوفى" أو "قتيل" = High severity مباشرة!

أعطني JSON فقط بدون أي نص إضافي."""

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=300
        )
        
        result_text = response.choices[0].message.content.strip()
        
        if result_text.startswith('```json'):
            result_text = result_text[7:]
        if result_text.endswith('```'):
            result_text = result_text[:-3]
        result_text = result_text.strip()
        
        result = json.loads(result_text)
        
        return result
        
    except Exception as e:
        print(f"LLM Error: {e}")
        return None

def classify_rule_based(text):
    text_lower = text.lower()
    
    if len(text.strip()) < 5:
        return {
            'severity': 'Unclear',
            'confidence': 0.30,
            'type': 'بلاغ غير مفهوم',
            'category': 'غير واضح',
            'action': 'unclear'
        }
    
    death_keywords = ['وفاة', 'وفيات', 'متوفى', 'قتيل', 'موت', 'توفي', 'مات']
    has_death = any(word in text_lower for word in death_keywords)
    if has_death:
        return {
            'severity': 'High',
            'confidence': 0.95,
            'type': 'بلاغ وفاة - يتطلب مباشرة فورية',
            'category': 'وفيات',
            'action': 'emergency_field'
        }
    
    high_emergency = {
        'اختطاف': 'اعتداء',
        'قتل': 'اعتداء',
        'طعن': 'اعتداء',
        'إطلاق نار': 'أمنية',
        'اطلاق نار': 'أمنية',
        'مسلح': 'أمنية',
        'انفجار': 'أمنية',
        'حريق': 'حرائق',
        'اتجار بالبشر': 'اتجار',
        'اتجار': 'اتجار'
    }
    
    for keyword, cat in high_emergency.items():
        if keyword in text_lower:
            return {
                'severity': 'High',
                'confidence': 0.92,
                'type': 'بلاغ يتطلب مباشرة أمنية ميدانية',
                'category': cat,
                'action': 'emergency_field'
            }
    
    electronic_station = {
        'تحرش': 'تحرش',
        'ابتزاز': 'معلوماتية',
        'احتيال': 'معلوماتية',
        'قرصنة': 'معلوماتية',
        'نصب': 'معلوماتية'
    }
    
    for keyword, cat in electronic_station.items():
        if keyword in text_lower:
            return {
                'severity': 'High',
                'confidence': 0.88,
                'type': 'تقديم بلاغ إلكتروني بمركز الشرطة',
                'category': cat,
                'action': 'electronic_report'
            }
    
    medium_incidents = {
        'مشاجرة': 'اعتداء',
        'مضاربة': 'اعتداء',
        'ضرب': 'اعتداء',
        'اعتداء': 'اعتداء',
        'سرقة': 'أمنية',
        'سطو': 'أمنية',
        'حادث': 'حوادث',
        'تصادم': 'حوادث',
        'سرعة': 'مرورية',
        'تفحيط': 'مرورية',
        'عكس السير': 'مرورية',
        'دهس': 'مرورية',
        'متهور': 'مرورية'
    }
    
    for keyword, cat in medium_incidents.items():
        if keyword in text_lower:
            return {
                'severity': 'Medium',
                'confidence': 0.80,
                'type': 'بلاغ يتطلب دورية',
                'category': cat,
                'action': 'patrol_required'
            }
    
    routine_contact = {
        'متوقفة': 'مرورية',
        'موقفة': 'مرورية',
        'متعطلة': 'مرورية',
        'ضوضاء': 'عامة',
        'شكوى': 'شكوى',
        'إزعاج': 'عامة'
    }
    
    for keyword, cat in routine_contact.items():
        if keyword in text_lower:
            if cat == 'شكوى':
                return {
                    'severity': 'Normal',
                    'confidence': 0.75,
                    'type': 'شكوى',
                    'category': 'شكوى عامة',
                    'action': 'complaint'
                }
            else:
                return {
                    'severity': 'Normal',
                    'confidence': 0.75,
                    'type': 'بلاغ مرورية',
                    'category': 'بلاغات مرورية',
                    'action': 'contact_first'
                }
    
    return {
        'severity': 'Unclear',
        'confidence': 0.50,
        'type': 'بلاغ غير مفهوم',
        'category': 'يحتاج توضيح',
        'action': 'unclear'
    }

def get_action_summary(action):
    actions_map = {
        'emergency_field': '🚨 ميدانية فورية',
        'patrol_required': '⚡ دورية',
        'contact_first': '📞 تواصل',
        'electronic_report': '💻 إلكتروني',
        'complaint': '📋 شكوى',
        'unclear': '❓ غير مفهوم'
    }
    return actions_map.get(action, '📋 عام')

def get_simple_actions(action):
    if action == 'emergency_field':
        return """
        <div style='background:#fff; padding:20px; border-radius:10px; margin-top:15px; border:2px solid #dc3545;'>
            <h3 style='margin:0 0 15px 0; color:#dc3545; font-size:18px; text-align:center;'>📋 الإجراءات المطلوبة</h3>
            <div style='background:#f8d7da; padding:15px; border-radius:8px; border-right:4px solid #dc3545;'>
                <ul style='margin:0; padding-right:20px; line-height:2.2; color:#721c24; font-size:15px;'>
                    <li>إرسال أقرب دورية فوراً</li>
                    <li>إبلاغ غرفة العمليات</li>
                    <li>تأمين المكان</li>
                    <li>توثيق الحادث</li>
                </ul>
            </div>
        </div>
        """
    
    elif action == 'electronic_report':
        return """
        <div style='background:#fff; padding:20px; border-radius:10px; margin-top:15px; border:2px solid #17a2b8;'>
            <h3 style='margin:0 0 15px 0; color:#17a2b8; font-size:18px; text-align:center;'>📋 الإجراءات المطلوبة</h3>
            <div style='background:#d1ecf1; padding:15px; border-radius:8px; border-right:4px solid #17a2b8;'>
                <ul style='margin:0; padding-right:20px; line-height:2.2; color:#0c5460; font-size:15px;'>
                    <li>تسجيل البلاغ إلكترونياً</li>
                    <li>منح رقم مرجعي</li>
                    <li>إحالة للجهة المختصة</li>
                    <li>متابعة وإبلاغ المبلّغ</li>
                </ul>
            </div>
        </div>
        """
    
    elif action == 'patrol_required':
        return """
        <div style='background:#fff; padding:20px; border-radius:10px; margin-top:15px; border:2px solid #ffc107;'>
            <h3 style='margin:0 0 15px 0; color:#ffc107; font-size:18px; text-align:center;'>📋 الإجراءات المطلوبة</h3>
            <div style='background:#fff3cd; padding:15px; border-radius:8px; border-right:4px solid #ffc107;'>
                <ul style='margin:0; padding-right:20px; line-height:2.2; color:#856404; font-size:15px;'>
                    <li>توجيه أقرب دورية</li>
                    <li>المباشرة الميدانية</li>
                    <li>اتخاذ الإجراء المناسب</li>
                    <li>إعداد تقرير</li>
                </ul>
            </div>
        </div>
        """
    
    elif action == 'contact_first':
        return """
        <div style='background:#fff; padding:20px; border-radius:10px; margin-top:15px; border:2px solid #28a745;'>
            <h3 style='margin:0 0 15px 0; color:#28a745; font-size:18px; text-align:center;'>📋 الإجراءات المطلوبة</h3>
            <div style='background:#d4edda; padding:15px; border-radius:8px; border-right:4px solid #28a745;'>
                <ul style='margin:0; padding-right:20px; line-height:2.2; color:#155724; font-size:15px;'>
                    <li>التواصل مع المعني</li>
                    <li>منح مهلة (15-30 دقيقة)</li>
                    <li>إذا لم يستجب: إرسال دورية</li>
                </ul>
            </div>
        </div>
        """
    
    elif action == 'complaint':
        return """
        <div style='background:#fff; padding:20px; border-radius:10px; margin-top:15px; border:2px solid #6c757d;'>
            <h3 style='margin:0 0 15px 0; color:#6c757d; font-size:18px; text-align:center;'>📋 الإجراءات المطلوبة</h3>
            <div style='background:#e9ecef; padding:15px; border-radius:8px; border-right:4px solid #6c757d;'>
                <ul style='margin:0; padding-right:20px; line-height:2.2; color:#495057; font-size:15px;'>
                    <li>تسجيل الشكوى</li>
                    <li>إحالة للجهة المختصة</li>
                    <li>الرد على المشتكي</li>
                </ul>
            </div>
        </div>
        """
    
    elif action == 'unclear':
        return """
        <div style='background:#fff; padding:20px; border-radius:10px; margin-top:15px; border:2px solid #ffc107;'>
            <h3 style='margin:0 0 15px 0; color:#ffc107; font-size:18px; text-align:center;'>📋 الإجراءات المطلوبة</h3>
            <div style='background:#fff3cd; padding:15px; border-radius:8px; border-right:4px solid #ffc107;'>
                <ul style='margin:0; padding-right:20px; line-height:2.2; color:#856404; font-size:15px;'>
                    <li>التواصل مع المبلّغ</li>
                    <li>طلب توضيحات</li>
                    <li>إعادة التصنيف</li>
                </ul>
            </div>
        </div>
        """
    
    return ""

text_input = widgets.Textarea(value='', placeholder='اكتب البلاغ هنا...', description='', layout=widgets.Layout(width='100%', height='120px'))
output_area = widgets.Output()
stats_area = widgets.Output()

classify_btn = widgets.Button(description='🔍 تصنيف', button_style='success', layout=widgets.Layout(width='130px', height='45px'))
clear_btn = widgets.Button(description='🗑️ مسح', button_style='', layout=widgets.Layout(width='90px', height='45px'))
history_btn = widgets.Button(description='📊 السجل', button_style='info', layout=widgets.Layout(width='100px', height='45px'))
unclear_btn = widgets.Button(description='❓ غير مفهومة', button_style='warning', layout=widgets.Layout(width='120px', height='45px'))

def update_stats():
    with stats_area:
        clear_output()
        
        total = len(history)
        if total == 0:
            return
        
        clear_count = total - severity_stats['Unclear']
        unclear_count = severity_stats['Unclear']
        high_count = severity_stats['High']
        medium_count = severity_stats['Medium']
        normal_count = severity_stats['Normal']
        
        html = f"""
        <div style='display:grid; grid-template-columns: 1fr 1fr; gap:15px; margin-bottom:20px;'>
            <div style='background:linear-gradient(135deg, #28a745 0%, #20c997 100%); padding:25px; border-radius:12px; box-shadow:0 2px 8px rgba(0,0,0,0.1);'>
                <h3 style='color:white; margin:0 0 15px 0; font-size:16px;'>✅ بلاغات واضحة</h3>
                <div style='font-size:48px; font-weight:bold; color:white; text-align:center;'>{clear_count}</div>
                <div style='text-align:center; color:rgba(255,255,255,0.9); font-size:14px; margin-top:8px;'>
                    {(clear_count/total*100) if total > 0 else 0:.1f}% من الإجمالي
                </div>
            </div>
            <div style='background:linear-gradient(135deg, #ffc107 0%, #ff9800 100%); padding:25px; border-radius:12px; box-shadow:0 2px 8px rgba(0,0,0,0.1);'>
                <h3 style='color:white; margin:0 0 15px 0; font-size:16px;'>❓ بلاغات غير مفهومة</h3>
                <div style='font-size:48px; font-weight:bold; color:white; text-align:center;'>{unclear_count}</div>
                <div style='text-align:center; color:rgba(255,255,255,0.9); font-size:14px; margin-top:8px;'>
                    {(unclear_count/total*100) if total > 0 else 0:.1f}% من الإجمالي
                </div>
            </div>
        </div>
        
        <div style='background:#f8f9fa; padding:20px; border-radius:12px; margin-bottom:20px; border:1px solid #dee2e6;'>
            <h3 style='margin:0 0 20px 0; color:#495057; font-size:18px;'>📊 تقسيم البلاغات حسب الأهمية</h3>
            <div style='display:grid; grid-template-columns: repeat(3, 1fr); gap:15px;'>
                <div style='background:white; padding:20px; border-radius:10px; border-left:4px solid #dc3545; box-shadow:0 1px 4px rgba(0,0,0,0.05);'>
                    <h4 style='margin:0; color:#dc3545; font-size:15px;'>🔴 عالية الخطورة</h4>
                    <div style='font-size:36px; font-weight:bold; color:#dc3545; text-align:center; margin:15px 0;'>{high_count}</div>
                    <div style='background:#f8d7da; padding:8px; border-radius:6px; text-align:center; font-size:13px; color:#721c24;'>
                        {(high_count/clear_count*100) if clear_count > 0 else 0:.1f}% من الواضحة
                    </div>
                </div>
                <div style='background:white; padding:20px; border-radius:10px; border-left:4px solid #ffc107; box-shadow:0 1px 4px rgba(0,0,0,0.05);'>
                    <h4 style='margin:0; color:#ffc107; font-size:15px;'>🟡 متوسطة</h4>
                    <div style='font-size:36px; font-weight:bold; color:#ffc107; text-align:center; margin:15px 0;'>{medium_count}</div>
                    <div style='background:#fff3cd; padding:8px; border-radius:6px; text-align:center; font-size:13px; color:#856404;'>
                        {(medium_count/clear_count*100) if clear_count > 0 else 0:.1f}% من الواضحة
                    </div>
                </div>
                <div style='background:white; padding:20px; border-radius:10px; border-left:4px solid #28a745; box-shadow:0 1px 4px rgba(0,0,0,0.05);'>
                    <h4 style='margin:0; color:#28a745; font-size:15px;'>🟢 عادية</h4>
                    <div style='font-size:36px; font-weight:bold; color:#28a745; text-align:center; margin:15px 0;'>{normal_count}</div>
                    <div style='background:#d4edda; padding:8px; border-radius:6px; text-align:center; font-size:13px; color:#155724;'>
                        {(normal_count/clear_count*100) if clear_count > 0 else 0:.1f}% من الواضحة
                    </div>
                </div>
            </div>
        </div>
        
        <div style='background:#f8f9fa; padding:20px; border-radius:12px; border:1px solid #dee2e6;'>
            <h3 style='margin:0 0 20px 0; color:#495057; font-size:18px;'>📋 تقسيم حسب نوع الإجراء</h3>
            <div style='display:grid; grid-template-columns: repeat(3, 1fr); gap:12px;'>
                <div style='background:white; padding:15px; border-radius:8px; text-align:center;'>
                    <div style='font-size:24px; font-weight:bold; color:#dc3545;'>{stats['emergency_field']}</div>
                    <div style='font-size:12px; color:#666; margin-top:5px;'>🚨 ميدانية فورية</div>
                </div>
                <div style='background:white; padding:15px; border-radius:8px; text-align:center;'>
                    <div style='font-size:24px; font-weight:bold; color:#ffc107;'>{stats['patrol_required']}</div>
                    <div style='font-size:12px; color:#666; margin-top:5px;'>⚡ دورية</div>
                </div>
                <div style='background:white; padding:15px; border-radius:8px; text-align:center;'>
                    <div style='font-size:24px; font-weight:bold; color:#28a745;'>{stats['contact_first']}</div>
                    <div style='font-size:12px; color:#666; margin-top:5px;'>📞 تواصل أولاً</div>
                </div>
                <div style='background:white; padding:15px; border-radius:8px; text-align:center;'>
                    <div style='font-size:24px; font-weight:bold; color:#17a2b8;'>{stats['electronic_report']}</div>
                    <div style='font-size:12px; color:#666; margin-top:5px;'>💻 إلكتروني</div>
                </div>
                <div style='background:white; padding:15px; border-radius:8px; text-align:center;'>
                    <div style='font-size:24px; font-weight:bold; color:#6c757d;'>{stats['complaint']}</div>
                    <div style='font-size:12px; color:#666; margin-top:5px;'>📋 شكاوى</div>
                </div>
                <div style='background:white; padding:15px; border-radius:8px; text-align:center;'>
                    <div style='font-size:24px; font-weight:bold; color:#495057;'>{total}</div>
                    <div style='font-size:12px; color:#666; margin-top:5px;'>📊 إجمالي</div>
                </div>
            </div>
        </div>
        """
        display(HTML(html))

def on_classify_click(b):
    with output_area:
        clear_output()
        
        text = text_input.value
        
        if not text.strip():
            display(HTML("<div style='background:#fff3cd; padding:15px; border-radius:8px; border-left:4px solid #ffc107;'><h4 style='color:#856404; margin:0;'>⚠️ تحذير</h4><p style='margin:5px 0 0 0;'>الرجاء إدخال نص البلاغ</p></div>"))
            return
        
        api_key = api_key_input.value.strip()
        classification_method = "Rule-Based"
        
        if api_key and openai_available:
            display(HTML("<div style='background:#d1ecf1; padding:10px; border-radius:6px; margin-bottom:10px; text-align:center; color:#0c5460;'>🤖 جاري التصنيف باستخدام LLM...</div>"))
            result = classify_with_llm(text, api_key)
            if result:
                classification_method = "LLM (GPT-4o-mini)"
            else:
                result = classify_rule_based(text)
                classification_method = "Rule-Based (Fallback)"
        else:
            result = classify_rule_based(text)
        
        report_entry = {
            'id': len(history) + 1,
            'text': text,
            'type': result['type'],
            'category': result['category'],
            'action': result['action'],
            'severity': result['severity'],
            'confidence': result['confidence'],
            'method': classification_method,
            'time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        history.append(report_entry)
        stats[result['action']] += 1
        severity_stats[result['severity']] += 1
        
        if result['action'] == 'unclear':
            unclear_reports.append(report_entry)
        
        severity_colors = {
            'High': {'bg': '#f8d7da', 'border': '#dc3545', 'text': '#721c24'},
            'Medium': {'bg': '#fff3cd', 'border': '#ffc107', 'text': '#856404'},
            'Normal': {'bg': '#d4edda', 'border': '#28a745', 'text': '#155724'},
            'Unclear': {'bg': '#fff3cd', 'border': '#ffc107', 'text': '#856404'}
        }
        
        color = severity_colors[result['severity']]
        conf_percent = result['confidence'] * 100
        
        method_badge = f"""
        <div style='display:inline-block; background:{'#667eea' if 'LLM' in classification_method else '#6c757d'}; color:white; padding:4px 12px; border-radius:15px; font-size:12px; margin-right:10px;'>
            {'🤖 LLM' if 'LLM' in classification_method else '📋 Keywords'}
        </div>
        """
        
        simple_actions = get_simple_actions(result['action'])
        
        html = f"""
        <div style='background:{color['bg']}; padding:25px; border-radius:12px; border-left:6px solid {color['border']}; margin-bottom:20px;'>
            <div style='background:white; padding:15px; border-radius:8px; margin-bottom:15px;'>
                <p style='margin:0; color:#666; font-size:13px;'><strong>📝 نص البلاغ:</strong></p>
                <p style='margin:8px 0 0 0; color:#333; font-size:16px;'>{text}</p>
            </div>
            
            <div style='margin-bottom:15px;'>
                {method_badge}
                <span style='color:{color['text']}; font-size:13px;'>طريقة التصنيف: {classification_method}</span>
            </div>
            
            <div style='display:grid; grid-template-columns: 1fr 1fr 1fr; gap:12px; margin-bottom:12px;'>
                <div style='background:rgba(255,255,255,0.9); padding:12px; border-radius:8px;'>
                    <p style='margin:0; color:#666; font-size:11px;'>التصنيف</p>
                    <h5 style='margin:5px 0 0 0; color:{color['text']}; font-size:13px;'>{result['severity']}</h5>
                </div>
                <div style='background:rgba(255,255,255,0.9); padding:12px; border-radius:8px;'>
                    <p style='margin:0; color:#666; font-size:11px;'>الفئة</p>
                    <h5 style='margin:5px 0 0 0; color:{color['text']}; font-size:13px;'>{result['category']}</h5>
                </div>
                <div style='background:rgba(255,255,255,0.9); padding:12px; border-radius:8px;'>
                    <p style='margin:0; color:#666; font-size:11px;'>مستوى الثقة</p>
                    <h5 style='margin:5px 0 0 0; color:{color['text']}; font-size:13px;'>{conf_percent:.0f}%</h5>
                </div>
            </div>
            <div style='background:{color['text']}; color:white; padding:10px 15px; border-radius:6px; text-align:center; font-size:13px;'>
                <strong>رقم البلاغ:</strong> #{report_entry['id']} | <strong>الإجراء:</strong> {get_action_summary(result['action'])}
            </div>
        </div>
        
        {simple_actions}
        """
        
        display(HTML(html))
        update_stats()

def on_clear_click(b):
    text_input.value = ''
    with output_area:
        clear_output()

def on_history_click(b):
    with output_area:
        clear_output()
        if not history:
            display(HTML("<div style='padding:30px; text-align:center; color:#666; background:#f8f9fa; border-radius:8px;'>📭 لا يوجد سجل بلاغات</div>"))
            return
        rows = ""
        for item in reversed(history[-20:]):
            action_emoji = {'emergency_field': '🚨', 'patrol_required': '⚡', 'contact_first': '📞', 'electronic_report': '💻', 'complaint': '📋', 'unclear': '❓'}
            bg_color = '#fff3cd' if item['action'] == 'unclear' else 'white'
            method_badge = '🤖' if 'LLM' in item.get('method', 'Rule-Based') else '📋'
            rows += f"<tr style='background:{bg_color};'><td style='padding:12px; border-bottom:1px solid #dee2e6; font-weight:bold;'>#{item['id']}</td><td style='padding:12px; border-bottom:1px solid #dee2e6;'>{item['text'][:50]}{'...' if len(item['text']) > 50 else ''}</td><td style='padding:12px; border-bottom:1px solid #dee2e6; text-align:center;'>{action_emoji[item['action']]} {get_action_summary(item['action'])}</td><td style='padding:12px; border-bottom:1px solid #dee2e6; text-align:center;'>{method_badge}</td><td style='padding:12px; border-bottom:1px solid #dee2e6; text-align:center;'>{item['confidence']*100:.0f}%</td><td style='padding:12px; border-bottom:1px solid #dee2e6; text-align:center; font-size:12px;'>{item['time']}</td></tr>"
        html = f"<div style='background:white; padding:20px; border-radius:10px; border:1px solid #dee2e6;'><h3 style='margin:0 0 15px 0; color:#495057;'>📊 سجل البلاغات (آخر 20)</h3><div style='overflow-x:auto;'><table style='width:100%; border-collapse:collapse;'><thead><tr style='background:#f8f9fa;'><th style='padding:12px; text-align:right; border-bottom:2px solid #dee2e6;'>#</th><th style='padding:12px; text-align:right; border-bottom:2px solid #dee2e6;'>البلاغ</th><th style='padding:12px; text-align:center; border-bottom:2px solid #dee2e6;'>الإجراء</th><th style='padding:12px; text-align:center; border-bottom:2px solid #dee2e6;'>الطريقة</th><th style='padding:12px; text-align:center; border-bottom:2px solid #dee2e6;'>الثقة</th><th style='padding:12px; text-align:center; border-bottom:2px solid #dee2e6;'>الوقت</th></tr></thead><tbody>{rows}</tbody></table></div><div style='margin-top:15px; padding:12px; background:#f8f9fa; border-radius:6px; text-align:center; font-size:13px; color:#666;'>إجمالي البلاغات: {len(history)}</div></div>"
        display(HTML(html))

def on_unclear_click(b):
    with output_area:
        clear_output()
        if not unclear_reports:
            display(HTML("<div style='padding:30px; text-align:center; color:#155724; background:#d4edda; border-radius:8px; border:2px solid #28a745;'><h3 style='margin:0; color:#155724;'>✅ ممتاز!</h3><p style='margin:8px 0 0 0;'>لا توجد بلاغات غير مفهومة</p></div>"))
            return
        rows = ""
        for item in reversed(unclear_reports):
            rows += f"<tr style='background:#fff3cd;'><td style='padding:12px; border-bottom:1px solid #dee2e6; font-weight:bold;'>#{item['id']}</td><td style='padding:12px; border-bottom:1px solid #dee2e6;'>{item['text']}</td><td style='padding:12px; border-bottom:1px solid #dee2e6; text-align:center;'>{item['confidence']*100:.0f}%</td><td style='padding:12px; border-bottom:1px solid #dee2e6; text-align:center; font-size:12px;'>{item['time']}</td></tr>"
        html = f"<div style='background:#fff3cd; padding:20px; border-radius:10px; border:2px solid #ffc107;'><h3 style='margin:0 0 15px 0; color:#856404;'>❓ البلاغات الغير مفهومة ({len(unclear_reports)})</h3><div style='background:rgba(0,0,0,0.05); padding:15px; border-radius:8px; margin-bottom:15px;'><h4 style='margin:0 0 10px 0; color:#856404;'>💡 الإجراءات الموصى بها:</h4><ul style='margin:0; padding-right:20px; line-height:2;'><li>مراجعة كل بلاغ يدوياً</li><li>التواصل مع المبلّغين للتوضيح</li><li>إعادة التصنيف بعد الحصول على المعلومات</li></ul></div><div style='overflow-x:auto;'><table style='width:100%; border-collapse:collapse; background:white; border-radius:8px; overflow:hidden;'><thead><tr style='background:#ffc107;'><th style='padding:12px; text-align:right; border-bottom:2px solid #dee2e6;'>#</th><th style='padding:12px; text-align:right; border-bottom:2px solid #dee2e6;'>نص البلاغ</th><th style='padding:12px; text-align:center; border-bottom:2px solid #dee2e6;'>الثقة</th><th style='padding:12px; text-align:center; border-bottom:2px solid #dee2e6;'>الوقت</th></tr></thead><tbody>{rows}</tbody></table></div></div>"
        display(HTML(html))

classify_btn.on_click(on_classify_click)
clear_btn.on_click(on_clear_click)
history_btn.on_click(on_history_click)
unclear_btn.on_click(on_unclear_click)

display(HTML("""<div style='background:linear-gradient(135deg, #1e5128 0%, #4e9f3d 100%); padding:30px; border-radius:15px; margin-bottom:20px; box-shadow:0 4px 15px rgba(0,0,0,0.2);'><h1 style='color:white; margin:0; text-align:center; font-size:32px;'>🇸🇦 كلنا أمن - نظام التصنيف الذكي</h1><p style='color:rgba(255,255,255,0.95); margin:10px 0 0 0; text-align:center; font-size:16px;'>يدعم التصنيف بالذكاء الاصطناعي (LLM)</p></div>"""))

if not openai_available:
    display(HTML("<div style='background:#fff3cd; padding:15px; border-radius:8px; margin-bottom:15px; border-left:4px solid #ffc107;'><strong>⚠️ تنبيه:</strong> مكتبة OpenAI غير مثبتة. قم بتثبيتها: <code>pip install openai</code></div>"))

display(HTML("<div style='margin:20px 0 10px 0;'><strong style='font-size:15px;'>🔑 OpenAI API Key (اختياري):</strong></div>"))
display(api_key_input)
display(HTML("<p style='margin:5px 0 15px 0; color:#666; font-size:13px;'>💡 إذا أدخلت API Key، سيتم استخدام GPT-4o-mini للتصنيف. وإلا سيستخدم النظام القائم على الكلمات المفتاحية.</p>"))

display(stats_area)
display(HTML("<div style='margin:20px 0 10px 0;'><strong style='font-size:15px;'>📝 أدخل نص البلاغ:</strong></div>"))
display(text_input)
display(widgets.HBox([classify_btn, clear_btn, history_btn, unclear_btn], layout=widgets.Layout(margin='12px 0')))
display(output_area)

Password(layout=Layout(width='100%'), placeholder='أدخل OpenAI API Key هنا (اختياري)')

Output()

Textarea(value='', layout=Layout(height='120px', width='100%'), placeholder='اكتب البلاغ هنا...')

Output()